#  **Data Augmentation by adding different kinds of noise** #

# White Noise (Low Level) #

In [2]:
import librosa
import soundfile as sf
from pydub import AudioSegment
import librosa
import os
import random
import numpy as np

In [4]:
import os
import random
import shutil

def sample_files(input_folder, output_folder, percentage):
    
    os.makedirs(output_folder, exist_ok=True)

    all_files = [f for f in os.listdir(input_folder) if os.path.isfile(os.path.join(input_folder, f))]

    sample_size = max(1, int(len(all_files) * (percentage / 100)))

    sampled_files = random.sample(all_files, sample_size)

    for file_name in sampled_files:
        src_path = os.path.join(input_folder, file_name)
        dst_path = os.path.join(output_folder, file_name)
        shutil.copy(src_path, dst_path)

    print(f"Sampled {len(sampled_files)} files out of {len(all_files)} and saved to {output_folder}.")

# Example usage
input_folder ="D:\\PD_AH"
output_folder = "D:\\Slow_PD"
sample_percentage = 50

sample_files(input_folder, output_folder, sample_percentage)

Sampled 20 files out of 40 and saved to D:\Slow_PD.


In [17]:

import os
import numpy as np
from scipy.io import wavfile

def add_noise_to_audio_folder(input_folder, noise_level, output_folder):
    
    for filename in os.listdir(input_folder):
        if filename.endswith('.wav'):
            input_file_path = os.path.join(input_folder, filename)
            sample_rate, data = wavfile.read(input_file_path)
            if len(data.shape) > 1:
                data = data.mean(axis=1)
            data = data.astype(np.float32) / np.max(np.abs(data))
            noise = np.random.normal(0, noise_level, data.shape)
            noisy_data = data + noise
            noisy_data = (noisy_data * 32767).astype(np.int16)
            output_file_path = os.path.join(output_folder, filename.replace('.wav', '.white_noise.wav'))
            wavfile.write(output_file_path, sample_rate, noisy_data)
            print(f"Noisy audio saved to: {output_file_path}")

input_folder ="D:\\HS_trim_1_4"
output_folder = "D:\\Phonation_new_Augmentations\\bts\\HS\\white"
noise_level = 0.05
add_noise_to_audio_folder(input_folder, noise_level, output_folder)



# Pink Noise (Low Level) #

In [18]:
import os
import numpy as np
from scipy.io import wavfile

def generate_pink_noise(size):
    white_noise = np.random.normal(0, 1, size)
    pink_noise = np.cumsum(white_noise)
    pink_noise = pink_noise / np.max(np.abs(pink_noise))
    return pink_noise

def add_pink_noise_to_audio_folder(input_folder, noise_level, output_folder):
    
    for filename in os.listdir(input_folder):
        if filename.endswith('.wav'):
            input_file_path = os.path.join(input_folder, filename)
            sample_rate, data = wavfile.read(input_file_path)
            if len(data.shape) > 1:
                data = data.mean(axis=1)
            data = data.astype(np.float32) / np.max(np.abs(data))
            pink_noise = generate_pink_noise(data.shape)
            pink_noise *= noise_level
            noisy_data = data + pink_noise
            noisy_data = np.clip(noisy_data, -1.0, 1.0)
            noisy_data = (noisy_data * 32767).astype(np.int16)
            output_file = os.path.join(output_folder, filename.replace('.wav', '.pink_noise.wav'))
            wavfile.write(output_file, sample_rate, noisy_data)
            print(f"Noisy audio (with pink noise) saved to: {output_file}")

input_folder = "D:\\HS_trim_1_4"
output_folder = "D:\\Phonation_new_Augmentations\\bts\\HS\\pink"
noise_level = 0.05
add_pink_noise_to_audio_folder(input_folder, noise_level, output_folder)



# Adding Impulse #

In [32]:

def add_impulse_noise_to_audio_folder(input_folder, impulse_density, impulse_amplitude, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for filename in os.listdir(input_folder):
        if filename.endswith('.wav'):
            input_file_path = os.path.join(input_folder, filename)
            sample_rate, data = wavfile.read(input_file_path)
            if len(data.shape) > 1:
                data = data.mean(axis=1)
            data = data.astype(np.float32) / np.max(np.abs(data))
            num_samples = len(data)
            impulse_mask = np.random.rand(num_samples) < impulse_density
            impulse_noise = np.zeros_like(data)
            impulse_noise[impulse_mask] = np.random.choice(
                [-impulse_amplitude, impulse_amplitude], size=np.sum(impulse_mask)
            )
            noisy_data = data + impulse_noise
            noisy_data = np.clip(noisy_data, -1.0, 1.0)
            noisy_data = (noisy_data * 32767).astype(np.int16)
            output_file = os.path.join(output_folder, filename.replace('.wav', '.impulse_noise.wav'))
            wavfile.write(output_file, sample_rate, noisy_data)
            print(f"Noisy audio (with impulse noise) saved to: {output_file}")

input_folder = "D:\\bts_random\PD"
output_folder = "D:\\Random_Augmentation\\bts_Aug\PD_bts_impulse"

impulse_density = 0.01  # Fraction of samples with impulses 
impulse_amplitude = 0.5  # Amplitude of the impulses 
add_impulse_noise_to_audio_folder(input_folder, impulse_density, impulse_amplitude, output_folder)


Noisy audio (with impulse noise) saved to: D:\Random_Augmentation\bts_Aug\PD_bts_impulse\PD1.bts.impulse_noise.wav
Noisy audio (with impulse noise) saved to: D:\Random_Augmentation\bts_Aug\PD_bts_impulse\PD2.bts.impulse_noise.wav


# Room Reverberation Effects #

In [33]:

from scipy.signal import convolve

def add_room_based_noise_to_audio_folder(input_folder, ir_file, reverb_level, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    ir_sample_rate, ir_data = wavfile.read(ir_file)
    if len(ir_data.shape) > 1:
        ir_data = ir_data.mean(axis=1)
    ir_data = ir_data.astype(np.float32) / np.max(np.abs(ir_data))
    for filename in os.listdir(input_folder):
        if filename.endswith('.wav'):
            input_file_path = os.path.join(input_folder, filename)
            sample_rate, data = wavfile.read(input_file_path)
            if len(data.shape) > 1:
                data = data.mean(axis=1)
            data = data.astype(np.float32) / np.max(np.abs(data))
            convolved_data = convolve(data, ir_data, mode='full')
            convolved_data *= reverb_level
            noisy_data = data + convolved_data[:len(data)]
            noisy_data = np.clip(noisy_data, -1.0, 1.0)
            noisy_data = (noisy_data * 32767).astype(np.int16)
            output_file = os.path.join(output_folder, filename.replace('.wav', '.room_reverb.wav'))
            wavfile.write(output_file, sample_rate, noisy_data)
            print(f"Audio with room-based noise (reverberation) saved to: {output_file}")

input_folder = "D:\\bts_random\PD"
output_folder = "D:\\Random_Augmentation\\bts_Aug\PD_bts_room"
ir_file = "C://Users//HP//Downloads//489765__pedrojuanupb__silentroom.wav"

reverb_level = 0.5  # Adjust reverb intensity
add_room_based_noise_to_audio_folder(input_folder, ir_file, reverb_level, output_folder)



C:\Users\HP\AppData\Local\Temp\ipykernel_17800\3003321044.py:5: WavFileWarning: Chunk (non-data) not understood, skipping it.
  ir_sample_rate, ir_data = wavfile.read(ir_file)


Audio with room-based noise (reverberation) saved to: D:\Random_Augmentation\bts_Aug\PD_bts_room\PD1.bts.room_reverb.wav
Audio with room-based noise (reverberation) saved to: D:\Random_Augmentation\bts_Aug\PD_bts_room\PD2.bts.room_reverb.wav


In [6]:
import os
import librosa
import soundfile as sf

def speed_up_audio_files(input_folder, output_folder, rate=1.1):
    os.makedirs(output_folder, exist_ok=True)
    for file_name in os.listdir(input_folder):
        if file_name.lower().endswith('.wav'):
            input_path = os.path.join(input_folder, file_name)
            # Load audio using soundfile to avoid soxr_ext issues
            y, sr = sf.read(input_path)
            # If audio has more than one channel, convert to mono by averaging channels
            if len(y.shape) > 1:
                y = np.mean(y, axis=1)
            # Time-stretch the audio using librosa (this operates on the numpy array)
            y_sped = librosa.effects.time_stretch(y, rate)
            base_name = os.path.splitext(file_name)[0]
            output_path = os.path.join(output_folder, base_name + ".spedup.wav")
            sf.write(output_path, y_sped, sr)
            print(f"Processed {file_name} -> {output_path}")


# Example usage:
input_folder = "D:\\Speed_files_HS"
output_folder_speedup = "D:\\Speed_HC"
speed_up_audio_files(input_folder, output_folder_speedup, rate=1.1)


TypeError: time_stretch() takes 1 positional argument but 2 were given

In [1]:
import os
import librosa
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances

ImportError: DLL load failed while importing _cext: The specified module could not be found.